# Initiation

In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup 
import random
import time
import pandas
from tqdm import tqdm
from datetime import datetime
from util import *
from glob import glob

In [2]:
def base_url_created (keyword,since,until):
    keykata = keyword.split(' ')
    keydate = "%20until%3A{}%20since%3A{}".format(until,since)
    if len(keykata) >=2:
        keysearch = '%20'.join(keykata)
        link = "https://twitter.com/search?lang=id&q={}{}&src=typed_query".format(keysearch,keydate)
    else:
        link = "https://twitter.com/search?lang=id&q={}{}&src=typed_query".format(keyword,keydate)
    
    return link

In [3]:
option = Options()
driver = webdriver.Chrome(options=option, executable_path=r'../chromedriver.exe')

# Search Keyword

In [4]:
keyword = "covid19"
since = "2020-5-2"
until = "2020-5-9"
print(base_url_created(keyword, since, until))
driver.get(base_url_created(keyword, since, until))

https://twitter.com/search?lang=id&q=covid19%20until%3A2020-5-9%20since%3A2020-5-2&src=typed_query


# Scroll Down

In [5]:
pages = 5
last_height = driver.execute_script("return document.body.scrollHeight")
for j in range(pages):
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # Wait to load page
    rand = random.randint(1,5)
    time.sleep(rand)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# Get List Post

In [6]:
soup = BeautifulSoup(driver.page_source, 'lxml')
post=soup.find_all("div",{"class":"css-1dbjc4n","data-testid":"tweet"},recursive=True)
len(post)

13

# Get Detail From Post

In [7]:
for i in (range(len(post))):
    time.sleep(4)
    print("Get Detail From {} Post".format(str(i+1)))
    df0 = pandas.DataFrame({"post":[],
                            "comment": [],
                            "parrent_comment":[]})
    
    caption, url_post = get_post(post[i])
    
    driver.get(url_post)
    pages = 10
    
    last_height = driver.execute_script("return document.body.scrollHeight")
    for j in range(pages):
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Wait to load page
        rand = random.randint(5,10)
        time.sleep(rand)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        
    soup = BeautifulSoup(driver.page_source, 'lxml')

    df0 = df0.append(get_comment(soup, caption, df0))
    
    df0.to_excel("data/covid19/{}-{}.xlsx".format(keyword, i))
    print("Saved Into Excel File")
    
    driver.execute_script("window.history.go(-1)")
        
driver.close()

Get Detail From 1 Post
Saved Into Excel File
Get Detail From 2 Post
Saved Into Excel File
Get Detail From 3 Post
Saved Into Excel File
Get Detail From 4 Post
Saved Into Excel File
Get Detail From 5 Post
Saved Into Excel File
Get Detail From 6 Post
Saved Into Excel File
Get Detail From 7 Post
Saved Into Excel File
Get Detail From 8 Post
Saved Into Excel File
Get Detail From 9 Post
Saved Into Excel File
Get Detail From 10 Post
Saved Into Excel File
Get Detail From 11 Post
Saved Into Excel File
Get Detail From 12 Post
Saved Into Excel File
Get Detail From 13 Post
Saved Into Excel File


# Merging XLSX

In [11]:
data = pandas.DataFrame()
for file in glob("data/covid19/*.xlsx"):
    new_data = pandas.read_excel(file)
    data = data.append(new_data)
data.to_excel("../../Emotion-Indo/covid/merged-covid19-twitter.xlsx")